## Importando o Pandas

In [1]:
import pandas as pd

## Carregando dados

In [2]:
dados = pd.read_csv('dados2.csv')
dados.head()

,CONCLIENTE,dataNascimentoCliente,sexoCliente,CONINSCRICAO,CONPRODUTO,periodicidadeCobranca,parcelamento,CONITEMCONTRATADO,codigoSistemaItemContratado,dataInicioVigenciaCobertura,...,prazoDecrescimoItemContratado,prazoCertoRendaItemContratado,prazoMinimoGarantidoRendaItemContratado,indiceReajusteItemContratado,valorCapitalSeguradoItemContratadoInicial,CONITEMCONTRATADOBENEFICIARIO,percentualParticipacaoBeneficio,CONBENEFICIARIO,dataNascimentoBeneficiario,sexoBeneficiario
0,1750926,1977-10-06,M,16013091,895,MENSAL,1,94352363,203045,2022-01-06,...,NaN,NaN,NaN,IPCA,100000.00,1840596.0,30.0,3616260.0,2006-04-14,FEMININO
1,1750926,1977-10-06,M,16013091,895,MENSAL,1,94352363,203045,2022-01-06,...,NaN,NaN,NaN,IPCA,100000.00,1840697.0,40.0,3616259.0,1976-05-29,FEMININO
2,1750926,1977-10-06,M,16013091,895,MENSAL,1,94352363,203045,2022-01-06,...,NaN,NaN,NaN,IPCA,100000.00,1841208.0,30.0,3616261.0,1997-11-25,MASCULINO
3,4300961,1957-04-15,M,16010702,895,MENSAL,1,94352382,203045,2022-01-06,...,NaN,NaN,NaN,IPCA,50000.00,1841180.0,100.0,3475459.0,1965-09-15,NÃO INFORMADO
4,3073128,1964-11-15,M,16013359,895,MENSAL,1,94352411,203045,2022-01-11,...,NaN,NaN,NaN,IPCA,342179.29,NaN,NaN,NaN,NaN,NaN


## Separando os dados

In [3]:
cliente = pd.read_csv('dados2.csv', usecols= [0,1,2]).rename(columns= {'CONCLIENTE': 'id', 'dataNascimentoCliente': 'data_nascimento', 'sexoCliente': 'sexo'})
cliente = cliente[~cliente['id'].isna()]
cliente.head()

inscricao = dados.iloc[:,[3, 0, 4, 5]]
inscricao.columns = ['id', 'cliente_id', 'produto_id', 'periodicidade']
inscricao.head()

itemcontratado = dados[['CONITEMCONTRATADO', 'CONINSCRICAO', 'codigoSistemaItemContratado', 'dataInicioVigenciaCobertura', 'prazoCobertura', 'prazoContribuicao', 'prazoCertoRendaItemContratado', 'prazoMinimoGarantidoRendaItemContratado', 'prazoDecrescimoItemContratado', 'indiceReajusteItemContratado', 'valorCapitalSeguradoItemContratadoInicial']]
itemcontratado.columns = ['id', 'inscricao_id', 'itemproduto_id', 'data_assinatura', 'prazo_cobertura', 'prazo_pagamento', 'prazo_renda', 'prazo_certo_renda', 'prazo_decrescimo', 'indice_reajuste', 'beneficio_inicial']
itemcontratado = itemcontratado[~itemcontratado['id'].isna()]
itemcontratado.head()

itembeneciario = dados[['CONITEMCONTRATADOBENEFICIARIO', 'CONITEMCONTRATADO', 'CONBENEFICIARIO', 'percentualParticipacaoBeneficio']]
itembeneciario.columns = ['id', 'itemcontratado_id', 'beneficiario_id', 'percentual_beneficio']
itembeneciario = itembeneciario[~itembeneciario['id'].isna()]
itembeneciario.head()

beneficiario = dados[['CONBENEFICIARIO', 'dataNascimentoBeneficiario', 'sexoBeneficiario']]
beneficiario.columns = ['id', 'data_nascimento', 'sexo']
beneficiario = beneficiario[~beneficiario['id'].isna()]
beneficiario.head()

,id,data_nascimento,sexo
0,3616260.0,2006-04-14,FEMININO
1,3616259.0,1976-05-29,FEMININO
2,3616261.0,1997-11-25,MASCULINO
3,3475459.0,1965-09-15,NÃO INFORMADO
5,3615870.0,1983-05-21,MASCULINO


## Trabalhando com SQL

In [4]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table, inspect

In [5]:
engine = create_engine('sqlite:///dados.db')

cliente.to_sql('cliente', engine, index= False, if_exists= 'replace')
inscricao.to_sql('inscricao', engine, index= False, if_exists= 'replace')
itemcontratado.to_sql('itemcontratado', engine, index = False, if_exists= 'replace')
itembeneciario.to_sql('itembeneciario', engine, index = False, if_exists= 'replace')
beneficiario.to_sql('beneficiario', engine, index = False, if_exists= 'replace')


inspector = inspect(engine)
print(inspector.get_table_names())


['beneficiario', 'cliente', 'inscricao', 'itembeneciario', 'itemcontratado']


## Quantidade de Clientes e de beneficiários por sexo

In [6]:
query = """SELECT A.*, B.CONT_CLI FROM (SELECT CASE WHEN SEXO = "MASCULINO" THEN "M" 
                                                    WHEN SEXO = "FEMININO" THEN "F" 
                                                    ELSE "NA" END AS SEXO, 
                                               COUNT(*) AS CONT_BENEF 
                                        FROM BENEFICIARIO GROUP BY CASE WHEN SEXO = "MASCULINO" THEN "M" 
                                                                        WHEN SEXO = "FEMININO" THEN "F" ELSE "NA" END) A
           LEFT JOIN (SELECT SEXO, COUNT(*) AS CONT_CLI FROM CLIENTE GROUP BY SEXO) B ON A.SEXO = B.SEXO"""

pd.read_sql(query, engine)


,SEXO,CONT_BENEF,CONT_CLI
0,F,130,421.0
1,M,432,579.0
2,NA,202,NaN


## Clientes com mais inscricoes

In [7]:
query = """SELECT CLIENTE_ID, COUNT(*) AS QTD 
           FROM INSCRICAO 
           GROUP BY CLIENTE_ID
           HAVING QTD > 1
           ORDER BY COUNT(*) DESC"""
           
pd.read_sql(query, engine).style.highlight_max(color= 'lightblue', subset = 'QTD').highlight_min(color = 'lightyellow', subset = 'QTD')

,cliente_id,QTD
0,7583711,6
1,4740226,6
2,1997789,6
3,7619720,5
4,7551345,5
5,7523381,5
6,7505219,5
7,1632262,5
8,1228037,5
9,7617513,4


## Beneficiários dos dois clientes com mais beneficiários

In [31]:
query = """SELECT TEST.BENEFICIARIO_ID FROM (
                SELECT CLIENTE_ID, COUNT (DISTINCT BENEFICIARIO_ID) AS QTD_BENEF FROM (
                    SELECT B.CLIENTE_ID, C.BENEFICIARIO_ID 
                    FROM ITEMCONTRATADO A
                    LEFT JOIN INSCRICAO B ON A.INSCRICAO_ID = B.ID
                    LEFT JOIN ITEMBENECIARIO C ON A.ID = C.ITEMCONTRATADO_ID)
                GROUP BY CLIENTE_ID
                ORDER BY COUNT(BENEFICIARIO_ID) DESC
                LIMIT 2) TESTE
                INNER JOIN (SELECT DISTINCT B.CLIENTE_ID, C.BENEFICIARIO_ID 
                    FROM ITEMCONTRATADO A
                    LEFT JOIN INSCRICAO B ON A.INSCRICAO_ID = B.ID
                    LEFT JOIN ITEMBENECIARIO C ON A.ID = C.ITEMCONTRATADO_ID) TEST ON TESTE.CLIENTE_ID = TEST.CLIENTE_ID"""
pd.read_sql(query, engine)

,BENEFICIARIO_ID
0,3631309.0
1,3631310.0
2,3631311.0
3,3631312.0
4,3631313.0
5,3631314.0
6,3623815.0
7,3623816.0
8,3623817.0
9,3623818.0


## Classificação

In [37]:
query = """SELECT ID, CASE WHEN QUARTILE = 1 THEN "BAIXO"
                           WHEN QUARTILE = 2 THEN "MEDIO"
                           ELSE "ALTO" END AS CLASSIFICACAO
           FROM (SELECT *, NTILE(3) OVER(ORDER BY BENEFICIO_INICIAL DESC) AS Quartile 
                 FROM ITEMCONTRATADO)"""

pd.read_sql(query, engine)

,id,CLASSIFICACAO
0,95865268,BAIXO
1,95865268,BAIXO
2,95865268,BAIXO
3,95865268,BAIXO
4,95665996,BAIXO
...,...,...
995,95733163,ALTO
996,95826958,ALTO
997,95963417,ALTO
998,95137127,ALTO


## ESTATÍSTICAS POR CLIENTE

In [55]:
query = """SELECT C.CLIENTE_ID, AVG(C.BENEFICIO_INICIAL) AS MEDIA, 
                  MIN(C.BENEFICIO_INICIAL) AS MIN,
                  MAX(C.BENEFICIO_INICIAL) AS MAX
                  FROM (SELECT B.CLIENTE_ID, A.BENEFICIO_INICIAL FROM ITEMCONTRATADO A 
                          LEFT JOIN INSCRICAO B ON A.INSCRICAO_ID = B.ID) C
            GROUP BY C.CLIENTE_ID"""

pd.read_sql(query, engine)

,CLIENTE_ID,MEDIA,MIN,MAX
0,144775,400000.00,400000.00,400000.00
1,168499,20000.00,20000.00,20000.00
2,206183,500000.00,500000.00,500000.00
3,208669,52885.48,52885.48,52885.48
4,235463,300000.00,300000.00,300000.00
...,...,...,...,...
630,7620078,450000.00,450000.00,450000.00
631,7620378,3000000.00,3000000.00,3000000.00
632,7621256,400000.00,400000.00,400000.00
633,7621715,500000.00,500000.00,500000.00


## Filtro de clientes

In [59]:
query = """SELECT DISTINCT C.* 
           FROM ITEMCONTRATADO A 
           LEFT JOIN INSCRICAO B ON A.INSCRICAO_ID = B.ID
           LEFT JOIN CLIENTE C ON B.CLIENTE_ID = C.ID
           WHERE B.PERIODICIDADE = "MENSAL" AND DATA_ASSINATURA >= '2022-01-06' AND DATA_ASSINATURA <= '2022-01-10'
           """

pd.read_sql(query, engine)

,id,data_nascimento,sexo
0,1750926,1977-10-06,M
1,4300961,1957-04-15,M
2,7488418,1963-03-05,F
3,2700079,1966-05-20,F
4,7503154,1983-03-04,F
...,...,...,...
105,7598207,1988-12-09,F
106,7598014,1970-10-06,M
107,7598746,1990-03-29,M
108,2435127,1977-06-17,M
